In [12]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import re
import bcrypt
import pymysql
import datetime
import mysql.connector

class RegisterScreen:
    def __init__(self, root):
        self.root = root
        self.root.title("Registration")
        self.root.geometry("900x600")
        
        # Configure style first
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self._configure_styles()

        # Create navigation frame
        self.nav_frame = tk.Frame(root, bg='#0984e3', height=50)
        self.nav_frame.pack(side='top', fill='x')
        self._create_navigation_bar()

        # Main container
        self.main_frame = tk.Frame(root, bg='#ffffff')
        self.main_frame.place(relx=0.5, rely=0.52, anchor='center', width=500, height=550)

        # Form header
        self.header = tk.Label(self.main_frame, 
                             text="Create Account",
                             font=('Arial', 22, 'bold'),
                             bg='#ffffff',
                             fg='#2d3436')
        self.header.grid(row=0, column=0, columnspan=2, pady=(20, 10), padx=20, sticky='ew')

        # Field configuration
        fields = [
            ('Full Name', 'fullname'),
            ('Email', 'email'),
            ('Phone', 'phone'),
            ('Username', 'username'),
            ('Password', 'password'),
            ('Confirm Password', 'confirm_password')
        ]

        # Create form elements
        self.entries = {}
        for row, (label_text, field_name) in enumerate(fields, start=1):
            lbl = ttk.Label(self.main_frame, 
                          text=label_text + " *",
                          style='FormLabel.TLabel')
            lbl.grid(row=row, column=0, padx=20, pady=5, sticky='e')

            entry = ttk.Entry(self.main_frame, 
                            style='FormEntry.TEntry',
                            width=30)
            if 'password' in field_name:
                entry.config(show='•')
            entry.grid(row=row, column=1, padx=20, pady=5, sticky='w')
            self.entries[field_name] = entry

       
        # Register button
        btn_register = ttk.Button(self.main_frame,
                                text="Create Account",
                                command=self.register_user,
                                style='FormButton.TButton')
        btn_register.grid(row=len(fields)+2, column=0, columnspan=2, 
                         pady=20, padx=20, ipady=8, sticky='ew')

        # Existing account link
        ttk.Button(self.main_frame,
                 text="Already have an account? Sign In",
                 command=self.return_to_login,
                 style='FormLink.TButton').grid(row=len(fields)+3, column=0, 
                                              columnspan=2, pady=10)

    def _create_navigation_bar(self):
        # Navigation elements
        logo_label = tk.Label(self.nav_frame, 
                            text="FreshCart", 
                            font=('Arial', 20, 'bold'), 
                            fg='white', 
                            bg='#0984e3')
        logo_label.pack(side='left', padx=20)

        nav_buttons_frame = tk.Frame(self.nav_frame, bg='#0984e3')
        nav_buttons_frame.pack(side='right', padx=20)

        nav_buttons = [
            ('Home', self.go_home),
            ('Login', self.go_login),
            ('Contact', self.go_contact)
        ]

        for text, command in nav_buttons:
            btn = tk.Button(nav_buttons_frame,
                          text=text,
                          font=('Arial', 12),
                          fg='white',
                          bg='#0984e3',
                          activeforeground='#dfe6e9',
                          activebackground='#074b8f',
                          bd=0,
                          cursor='hand2',
                          command=command)
            btn.pack(side='left', padx=10)

    def _configure_styles(self):
        self.style.configure('FormLabel.TLabel',
                           foreground='#2d3436',
                           font=('Arial', 10, 'bold'),
                           background='#ffffff')

        self.style.configure('FormEntry.TEntry',
                           font=('Arial', 10),
                           padding=5,
                           bordercolor='#dfe6e9',
                           lightcolor='#dfe6e9',
                           darkcolor='#dfe6e9')
        self.style.map('FormEntry.TEntry',
                     bordercolor=[('focus', '#74b9ff')],
                     lightcolor=[('focus', '#74b9ff')],
                     darkcolor=[('focus', '#74b9ff')])

        self.style.configure('FormButton.TButton',
                           foreground='white',
                           background='#0984e3',
                           font=('Arial', 12, 'bold'),
                           borderwidth=0,
                           focuscolor='#74b9ff')
        self.style.map('FormButton.TButton',
                     background=[('active', '#74b9ff')])

        self.style.configure('FormLink.TButton',
                           foreground='#0984e3',
                           background='#ffffff',
                           font=('Arial', 9),
                           borderwidth=0)
        self.style.map('FormLink.TButton',
                     foreground=[('active', '#74b9ff')])

        self.style.configure('FormCheckbutton.TCheckbutton',
                           background='#ffffff',
                           foreground='#2d3436',
                           font=('Arial', 9),
                           indicatorsize=14,
                           indicatordepth=2)
        self.style.map('FormCheckbutton.TCheckbutton',
                     foreground=[('active', '#0984e3'), ('!active', '#2d3436')],
                     indicatorcolor=[('selected', '#0984e3'), ('!selected', '#ffffff')])

    # Validation methods
    def validate_email(self, email):
        pattern = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
        return re.match(pattern, email) is not None

    def validate_phone(self, phone):
        return re.match(r"^[0-9]{10}$", phone) is not None

    def validate_password(self, password):
        if len(password) < 8:
            return False, "Password must be at least 8 characters"
        if not re.search(r"[A-Z]", password):
            return False, "Password must contain at least one uppercase letter"
        if not re.search(r"[a-z]", password):
            return False, "Password must contain at least one lowercase letter"
        if not re.search(r"[0-9]", password):
            return False, "Password must contain at least one number"
        return True, ""

    def register_user(self):
        data = {field: entry.get() for field, entry in self.entries.items()}
        
        # Validation checks
        required_fields = ['fullname', 'email', 'phone', 'username', 'password']
        for field in required_fields:
            if not data.get(field):
                messagebox.showerror("Error", f"{field.replace('_', ' ').title()} is required!")
                return

        if not self.validate_email(data['email']):
            messagebox.showerror("Error", "Invalid email address format!")
            return

        if not self.validate_phone(data['phone']):
            messagebox.showerror("Error", "Phone number must be 10 digits!")
            return

        valid_pass, pass_msg = self.validate_password(data['password'])
        if not valid_pass:
            messagebox.showerror("Error", pass_msg)
            return

        if data['password'] != data['confirm_password']:
            messagebox.showerror("Error", "Passwords do not match!")
            return

        

        # Database operations
        try:
            conn = connect_db()
            cursor = conn.cursor()
            
            cursor.execute("SELECT username FROM users WHERE username = %s", (data['username'],))
            if cursor.fetchone():
                messagebox.showerror("Error", "Username already exists!")
                return

            hashed_pw = bcrypt.hashpw(data['password'].encode(), bcrypt.gensalt())
            cursor.execute("""
                INSERT INTO users 
                (username, password, full_name, email, phone)
                VALUES (%s, %s, %s, %s, %s)
            """, (
                data['username'],
                hashed_pw,
                data['fullname'],
                data['email'],
                data['phone']
            ))
            
            conn.commit()
            messagebox.showinfo("Success", "Registration successful!\nYou can now login.")
            self.root.destroy()
            root = tk.Tk()
            LoginScreen(root, "user")
            root.mainloop()

        except pymysql.Error as e:
            messagebox.showerror("Database Error", f"Registration failed: {str(e)}")
            conn.rollback()
        finally:
            if conn and conn.open:
                conn.close()

    # Navigation methods
    def go_home(self):
        self.root.destroy()
        root = tk.Tk()
        FrontScreen(root)
        root.mainloop()

    def go_login(self):
        self.root.destroy()
        root = tk.Tk()
        LoginScreen(root, "user")
        root.mainloop()

    def go_contact(self):
        messagebox.showinfo("Contact Us", 
                          "Email: grocery_store.com\nPhone: 1-800-123-4567\nAddress: 123 Grocery Street")

    def return_to_login(self):
        self.root.destroy()
        root = tk.Tk()
        LoginScreen(root, "user")
        root.mainloop()

def connect_db():
    return pymysql.connect(
        host="localhost",
        user="root",          # Your MySQL username
        password="",          # Your MySQL password
        database="grocery",
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )


class FrontScreen:
    def __init__(self, root):
        self.root = root
        self.root.title("Grocery Store")
        self.root.geometry("800x500")
        self.bg_image = Image.open("C:/Users/YASHASWINI M/Downloads/gro.jpg")  
        self.bg_image = self.bg_image.resize((900, 900), Image.LANCZOS)
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)

        # Create background label
        self.bg_label = tk.Label(root, image=self.bg_photo)
        self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        
        # Add front screen implementation
        ttk.Label(root, text="Welcome to Grocery store", font=("Arial", 24)).pack(pady=50)
        ttk.Label(root, text="Fresh Groceries, Smarter Shopping! ", font=("Arial", 16)).pack(pady=40)
        ttk.Button(root, text="Start Shopping", command=self.open_registration).pack(pady=20)

    def open_registration(self):
        self.root.destroy()
        root = tk.Tk()
        RegisterScreen(root)
        root.mainloop()

class LoginScreen:
    def __init__(self, root, role):
        self.root = root
        self.role = role
        self.root.title(f"Login - {role.capitalize()}")
        self.root.geometry("400x300")

        self.bg_image = Image.open("C:/Users/YASHASWINI M/Downloads/gro.jpg")  
        self.bg_image = self.bg_image.resize((800, 900), Image.LANCZOS)
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)

        # Create background label
        self.bg_label = tk.Label(root, image=self.bg_photo)
        self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        
        frame = tk.Frame(root, bg="white", bd=5)
        frame.place(relx=0.5, rely=0.5, anchor="center")

        ttk.Label(frame, text=f"Login as {role.capitalize()}", font=("Arial", 18, "bold")).pack(pady=10)
        ttk.Label(frame, text="Username:").pack()
        self.username_var = tk.StringVar()
        ttk.Entry(frame, textvariable=self.username_var).pack()

        ttk.Label(frame, text="Password:").pack()
        self.password_var = tk.StringVar()
        ttk.Entry(frame, textvariable=self.password_var, show="*").pack()

        ttk.Button(frame, text="Login", command=self.authenticate).pack(pady=10)



    # In LoginScreen class
    def authenticate(self):
        username = self.username_var.get()
        password = self.password_var.get()
    
        try:
            conn = connect_db()
            with conn.cursor() as cursor:
            # Get stored password hash
                cursor.execute("SELECT password, role FROM users WHERE username = %s", (username,))
                result = cursor.fetchone()
            
                if result and bcrypt.checkpw(password.encode(), result['password'].encode()):
                    self.root.destroy()
                    if result['role'] == "admin":
                        root = tk.Tk()
                        AdminDashboard(root)
                        root.mainloop()
                    else:
                        root = tk.Tk()
                        UserPanel(root)
                        root.mainloop()
                else:
                    messagebox.showerror("Error", "Invalid username or password")
        except pymysql.Error as e:
            messagebox.showerror("Database Error", str(e))
        finally:
            if conn:
                conn.close()

    def open_administration(self):
        
        root = tk.Tk()
        AdminDashboard(root)
        root.mainloop()


class AdminDashboard:
    def __init__(self, root):
        self.root = root
        self.root.title("Admin Dashboard")
        self.root.geometry("1000x600")

        # Database connection
        self.db = self.connect_db()
        self.create_tables()

        # Create notebook (tabs)
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(pady=10, expand=True, fill='both')

        # Create frames for tabs
        self.user_frame = ttk.Frame(self.notebook)
        self.product_frame = ttk.Frame(self.notebook)

        # Add tabs to notebook
        self.notebook.add(self.user_frame, text='User Management')
        self.notebook.add(self.product_frame, text='Product Management')

        # Setup user and product management UI
        self.setup_user_management()
        self.setup_product_management()

    def connect_db(self):
        try:
            return mysql.connector.connect(
                host="localhost",
                user="root",
                password="",
                database="grocery"
            )
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error connecting to database: {err}")
            return None

    def create_tables(self):
        if self.db is not None:
            cursor = self.db.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS users (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    username VARCHAR(255) NOT NULL,
                    email VARCHAR(255) NOT NULL,
                    role VARCHAR(255) NOT NULL
                )
            """)
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS products (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(255) NOT NULL,
                    price DECIMAL(10,2) NOT NULL,
                    stock INT NOT NULL
                )
            """)
            self.db.commit()
            cursor.close()

    def setup_user_management(self):
        # User Treeview
        columns = ('id', 'username', 'password', 'role')
        self.user_tree = ttk.Treeview(self.user_frame, columns=columns, show='headings')
        self.user_tree.heading('id', text='ID')
        self.user_tree.heading('username', text='Username')
        self.user_tree.heading('password', text='password')
        self.user_tree.heading('role', text='Role')
        self.user_tree.pack(fill='both', expand=True, padx=10, pady=10)

        # Scrollbar
        scrollbar = ttk.Scrollbar(self.user_frame, orient=tk.VERTICAL, command=self.user_tree.yview)
        self.user_tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Control Frame
        control_frame = ttk.Frame(self.user_frame)
        control_frame.pack(pady=10)

        # Entry variables
        self.username_var = tk.StringVar()
        self.email_var = tk.StringVar()
        self.role_var = tk.StringVar()

        # Entry fields
        ttk.Label(control_frame, text="Username:").grid(row=0, column=0, padx=5)
        ttk.Entry(control_frame, textvariable=self.username_var).grid(row=0, column=1, padx=5)
        ttk.Label(control_frame, text="Email:").grid(row=0, column=2, padx=5)
        ttk.Entry(control_frame, textvariable=self.email_var).grid(row=0, column=3, padx=5)
        ttk.Label(control_frame, text="Role:").grid(row=0, column=4, padx=5)
        ttk.Entry(control_frame, textvariable=self.role_var).grid(row=0, column=5, padx=5)

        # Buttons
        ttk.Button(control_frame, text="Add User", command=self.add_user).grid(row=0, column=6, padx=5)
        ttk.Button(control_frame, text="Update User", command=self.update_user).grid(row=0, column=7, padx=5)
        ttk.Button(control_frame, text="Delete User", command=self.delete_user).grid(row=0, column=8, padx=5)

        self.update_user_tree()

    def setup_product_management(self):
        # Product Treeview
        columns = ('id', 'name','catogory','price', 'stock')
        self.product_tree = ttk.Treeview(self.product_frame, columns=columns, show='headings')
        self.product_tree.heading('id', text='ID')
        self.product_tree.heading('name', text='Product Name')
        self.product_tree.heading('catogory', text='Catogory')
        self.product_tree.heading('price', text='Price')
        self.product_tree.heading('stock', text='stock')
        
        self.product_tree.pack(fill='both', expand=True, padx=10, pady=10)

        # Scrollbar
        scrollbar = ttk.Scrollbar(self.product_frame, orient=tk.VERTICAL, command=self.product_tree.yview)
        self.product_tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Control Frame
        control_frame = ttk.Frame(self.product_frame)
        control_frame.pack(pady=10)

        # Entry variables
        self.product_name_var = tk.StringVar()
        self.price_var = tk.DoubleVar()
        self.stock_var = tk.IntVar()

        # Entry fields
        ttk.Label(control_frame, text="Product Name:").grid(row=0, column=0, padx=5)
        ttk.Entry(control_frame, textvariable=self.product_name_var).grid(row=0, column=1, padx=5)
        ttk.Label(control_frame, text="Price:").grid(row=0, column=2, padx=5)
        ttk.Entry(control_frame, textvariable=self.price_var).grid(row=0, column=3, padx=5)
        ttk.Label(control_frame, text="Stock:").grid(row=0, column=4, padx=5)
        ttk.Entry(control_frame, textvariable=self.stock_var).grid(row=0, column=5, padx=5)

        # Buttons
        ttk.Button(control_frame, text="Add Product", command=self.add_product).grid(row=0, column=6, padx=5)
        ttk.Button(control_frame, text="Update Product", command=self.update_product).grid(row=0, column=7, padx=5)
        ttk.Button(control_frame, text="Delete Product", command=self.delete_product).grid(row=0, column=8, padx=5)

        self.update_product_tree()

    # User management methods
    def add_user(self):
        if self.db is None:
            return

        query = "INSERT INTO users (username, email, role) VALUES (%s, %s, %s)"
        values = (
            self.username_var.get(),
            self.email_var.get(),
            self.role_var.get()
        )

        try:
            cursor = self.db.cursor()
            cursor.execute(query, values)
            self.db.commit()
            self.update_user_tree()
            self.clear_user_entries()
            messagebox.showinfo("Success", "User added successfully")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error adding user: {err}")
        finally:
            cursor.close()

    def update_user(self):
        selected = self.user_tree.selection()
        if not selected:
            return

        item = self.user_tree.item(selected[0])
        user_id = item['values'][0]

        query = """
            UPDATE users 
            SET username = %s, email = %s, role = %s 
            WHERE id = %s
        """
        values = (
            self.username_var.get(),
            self.email_var.get(),
            self.role_var.get(),
            user_id
        )

        try:
            cursor = self.db.cursor()
            cursor.execute(query, values)
            self.db.commit()
            self.update_user_tree()
            self.clear_user_entries()
            messagebox.showinfo("Success", "User updated successfully")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error updating user: {err}")
        finally:
            cursor.close()

    def delete_user(self):
        selected = self.user_tree.selection()
        if not selected:
            return

        user_id = self.user_tree.item(selected[0])['values'][0]

        try:
            cursor = self.db.cursor()
            cursor.execute("DELETE FROM users WHERE id = %s", (user_id,))
            self.db.commit()
            self.update_user_tree()
            self.clear_user_entries()
            messagebox.showinfo("Success", "User deleted successfully")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error deleting user: {err}")
        finally:
            cursor.close()

    def update_user_tree(self):
        for item in self.user_tree.get_children():
            self.user_tree.delete(item)

        try:
            cursor = self.db.cursor()
            cursor.execute("SELECT * FROM users")
            for row in cursor.fetchall():
                self.user_tree.insert('', tk.END, values=row)
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error loading users: {err}")
        finally:
            cursor.close()

    def clear_user_entries(self):
        self.username_var.set('')
        self.email_var.set('')
        self.role_var.set('')

    # Product management methods
    def add_product(self):
        if self.db is None:
            return

        query = "INSERT INTO products (name, price, stock) VALUES (%s, %s, %s)"
        values = (
            self.product_name_var.get(),
            self.price_var.get(),
            self.stock_var.get()
        )

        try:
            cursor = self.db.cursor()
            cursor.execute(query, values)
            self.db.commit()
            self.update_product_tree()
            self.clear_product_entries()
            messagebox.showinfo("Success", "Product added successfully")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error adding product: {err}")
        finally:
            cursor.close()

    def update_product(self):
        selected = self.product_tree.selection()
        if not selected:
            return

        item = self.product_tree.item(selected[0])
        product_id = item['values'][0]

        query = """
            UPDATE products 
            SET name = %s, price = %s, stock = %s 
            WHERE id = %s
        """
        values = (
            self.product_name_var.get(),
            self.price_var.get(),
            self.stock_var.get(),
            product_id
        )

        try:
            cursor = self.db.cursor()
            cursor.execute(query, values)
            self.db.commit()
            self.update_product_tree()
            self.clear_product_entries()
            messagebox.showinfo("Success", "Product updated successfully")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error updating product: {err}")
        finally:
            cursor.close()

    def delete_product(self):
        selected = self.product_tree.selection()
        if not selected:
            return

        product_id = self.product_tree.item(selected[0])['values'][0]

        try:
            cursor = self.db.cursor()
            cursor.execute("DELETE FROM products WHERE id = %s", (product_id,))
            self.db.commit()
            self.update_product_tree()
            self.clear_product_entries()
            messagebox.showinfo("Success", "Product deleted successfully")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error deleting product: {err}")
        finally:
            cursor.close()

    def update_product_tree(self):
        for item in self.product_tree.get_children():
            self.product_tree.delete(item)

        try:
            cursor = self.db.cursor()
            cursor.execute("SELECT * FROM products")
            for row in cursor.fetchall():
                self.product_tree.insert('', tk.END, values=row)
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error loading products: {err}")
        finally:
            cursor.close()

    def clear_product_entries(self):
        self.product_name_var.set('')
        self.price_var.set(0.0)
        self.stock_var.set(0)


class UserPanel:
    def __init__(self, root):
        self.root = root
        self.root.title("Grocery Shopping")
        self.root.geometry("1000x600")
        style = ttk.Style()
        style.configure("Accent.TButton", font=("Arial", 12), background="#3498db", foreground="white")
        style.configure("Danger.TButton", font=("Arial", 10), background="#e74c3c", foreground="white")
                
        self.cart = []  # Stores cart items
        
        # Database Connection
        self.conn = pymysql.connect(host='localhost', user='root', password='', database='grocery')
        self.cursor = self.conn.cursor()
        
        
        # Header Frame
        self.header = tk.Frame(root, bg="green", height=50)
        self.header.pack(fill=tk.X)
        
        tk.Label(self.header, text="Grocery Shop", font=("Arial", 18, "bold"), fg="white", bg="green").pack(side=tk.LEFT, padx=20)
        self.search_var = tk.StringVar()
        ttk.Entry(self.header, textvariable=self.search_var, width=40).pack(side=tk.LEFT, padx=10)
        ttk.Button(self.header, text="Search", command=self.search_product).pack(side=tk.LEFT)
        
        # Add Logout Button (styled with Danger.TButton) next to the cart button
        self.logout_btn = ttk.Button(
            self.header, 
            text="Logout", 
            command=self.logout,
            style="Blue.TButton"
        )
        self.logout_btn.pack(side=tk.RIGHT, padx=20)
        style.configure("Blue.TButton", font=("Arial", 10), foreground="blue")
        self.cart_btn = ttk.Button(self.header, text="View Cart", command=self.view_cart)
        self.cart_btn.pack(side=tk.RIGHT, padx=20)
        
       
        
        # Sidebar Frame for Categories
        self.sidebar = tk.Frame(root, bg="green", width=200)
        self.sidebar.pack(side=tk.LEFT, fill=tk.Y)
        
        self.cursor.execute("SELECT DISTINCT category FROM products")
        categories = [row[0] for row in self.cursor.fetchall()]
        
        tk.Label(self.sidebar, text="Categories", bg="gray", fg="white", font=("Arial", 14, "bold")).pack(fill=tk.X)
        for cat in categories:
            tk.Button(self.sidebar, text=cat, bg="white", font=("Arial", 12), command=lambda c=cat: self.show_products(c)).pack(fill=tk.X, pady=2)
        
        # Product Display Frame
        self.product_frame = tk.Frame(root, bg="white")
        self.product_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        if categories:
            self.show_products(categories[0])
    
    def show_products(self, category):
        for widget in self.product_frame.winfo_children():
            widget.destroy()
        
        self.cursor.execute("SELECT id, name, price FROM products WHERE category=%s", (category,))
        products = self.cursor.fetchall()
        
        tk.Label(self.product_frame, text=f"{category} Products", font=("Arial", 14, "bold"), bg="white").pack(pady=10)
        
        for product_id, product, price in products:
            frame = tk.Frame(self.product_frame, bg="lightgray", bd=2, relief=tk.RIDGE)
            frame.pack(pady=5, padx=20, fill=tk.X)
            
            tk.Label(frame, text=f"{product} - Rs. {price}", font=("Arial", 12)).pack(side=tk.LEFT, padx=10)
            
            quantity_var = tk.IntVar(value=1)
            quantity_dropdown = ttk.Combobox(frame, textvariable=quantity_var, values=[i for i in range(1, 21)], width=5)
            quantity_dropdown.pack(side=tk.LEFT, padx=10)
            
            tk.Button(frame, text="Add to Cart", command=lambda pid=product_id, p=product, pr=price, q=quantity_var: (self.add_to_cart(pid, p, pr, q),  
        messagebox.showinfo("Success", f"{p} added to cart!")
    )
).pack(side=tk.RIGHT, padx=10)
    def add_to_cart(self, product_id, product_name, price, quantity_var):
        self.cart.append((product_id, product_name, price, quantity_var.get()))  # 4 values
        
    def update_cart_quantity(self, product_id, new_quantity):
    # Update database first
        self.cursor.execute("""
            UPDATE cart 
            SET quantity = %s 
            WHERE user_id = %s AND product_id = %s
        """, (new_quantity, self.user_id, product_id))
        self.conn.commit()
        
        # Update local cart
        for idx, item in enumerate(self.cart):
            if item[0] == product_id:
                self.cart[idx] = (item[0], item[1], item[2], new_quantity)
        
        # Recalculate total
        total = sum(item[2] * item[3] for item in self.cart)
        self.total_var.set(f"₹{total:.2f}")

    def remove_from_cart(self, product_id, window):
        self.cursor.execute("""
            DELETE FROM cart 
            WHERE user_id = %s AND product_id = %s
        """, (self.user_id, product_id))
        self.conn.commit()
        
        # Remove from local cart
        self.cart = [item for item in self.cart if item[0] != product_id]
        
        # Refresh cart window
        window.destroy()
        self.view_cart()    
    
    def view_cart(self):
        if not self.cart:
            messagebox.showinfo("Cart", "Your cart is empty!")
            return

        cart_window = tk.Toplevel(self.root)
        cart_window.title("Cart")
        cart_window.geometry("400x400")
        
        ttk.Label(cart_window, text="Your Cart", font=("Arial", 14, "bold")).pack(pady=10)
        cart_list = tk.Listbox(cart_window, height=12, font=("Arial", 10))
        cart_list.pack(pady=5, fill=tk.BOTH, expand=True, padx=10)
    
        # Cart items and calculations
        subtotal = 0
        for item in self.cart:
            product_id, product_name, price, quantity = item   # Unpack the tuple
            item_total = price * quantity
            subtotal += item_total
            cart_list.insert(tk.END, f"{product_name} - Rs{price:.2f} x {quantity} = Rs{item_total:.2f}")
    
        # Tax and discount constants (define these in your class/config)
        TAX_RATE = 0.2       # 2% tax
        DISCOUNT_RATE = 0.10  # 10% discount
        DISCOUNT_THRESHOLD = 5000  # Rs.5000 threshold for discount
    
        # Calculate financials
        tax = subtotal * TAX_RATE
        discount = subtotal * DISCOUNT_RATE if subtotal > DISCOUNT_THRESHOLD else 0
        total = subtotal + tax - discount
    
        # Financial details frame
        details_frame = ttk.Frame(cart_window)
        details_frame.pack(pady=10, fill=tk.X, padx=10)
    
        ttk.Label(details_frame, text=f"Subtotal:", font=("Arial", 12)).grid(row=0, column=0, sticky=tk.W)
        ttk.Label(details_frame, text=f"Rs{subtotal:.2f}", font=("Arial", 12)).grid(row=0, column=1, sticky=tk.E)
    
        ttk.Label(details_frame, text=f"Tax ({TAX_RATE*100}%):", font=("Arial", 12)).grid(row=1, column=0, sticky=tk.W)
        ttk.Label(details_frame, text=f"Rs{tax:.2f}", font=("Arial", 12)).grid(row=1, column=1, sticky=tk.E)
    
        if discount > 0:
            ttk.Label(details_frame, text=f"Discount ({DISCOUNT_RATE*100}%):", font=("Arial", 12)).grid(row=2, column=0, sticky=tk.W)
            ttk.Label(details_frame, text=f"-Rs{discount:.2f}", font=("Arial", 12)).grid(row=2, column=1, sticky=tk.E)
    
        ttk.Label(details_frame, text="Total:", font=("Arial", 12, "bold")).grid(row=3, column=0, sticky=tk.W)
        ttk.Label(details_frame, text=f"Rs{total:.2f}", font=("Arial", 12, "bold")).grid(row=3, column=1, sticky=tk.E)
    
        # Generate bill button
        ttk.Button(cart_window, 
                  text="Generate Bill", 
                  command=lambda: self.generate_receipt(subtotal, tax, discount, total, cart_window)
                  ).pack(pady=10)
    
    def generate_receipt(self, subtotal, tax, discount, total, cart_window):
        # Create receipt content
        receipt_data = "\nGrocery Store Receipt\n"
        receipt_data += "=" * 40 + "\n"
        receipt_data += "Item                Qty    Price\n"
        receipt_data += "-" * 40 + "\n"
        
        for item in self.cart:
            product_id, product_name, price, quantity = item
            total_price = price * quantity
            receipt_data += f"{product_name:<18}{quantity:^7}Rs{total_price:>10.2f}\n"
        
        receipt_data += "=" * 40 + "\n"
        receipt_data += f"Subtotal: Rs{subtotal:.2f}\n"
        receipt_data += f"Tax: Rs{tax:.2f}\n"
        if discount > 0:
            receipt_data += f"Discount: -Rs{discount:.2f}\n"
        receipt_data += "-" * 40 + "\n"
        receipt_data += f"TOTAL: Rs{total:.2f}\n"
        receipt_data += "=" * 40 + "\n"
    
        # Generate timestamped filename
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"receipt_{timestamp}.txt"
        try:
            with open(filename, "w") as file:
                file.write(receipt_data)
        except IOError as e:
            messagebox.showerror("Error", f"Failed to save receipt: {e}")
            return
    
        # Show receipt in new window
        receipt_window = tk.Toplevel(self.root)
        receipt_window.title("Receipt")
        receipt_window.geometry("500x500")
        
        ttk.Label(receipt_window, text="Receipt Generated", font=("Arial", 16, "bold")).pack(pady=10)
        receipt_text = tk.Text(receipt_window, font=("Courier", 10), wrap=tk.WORD)
        receipt_text.insert(tk.END, receipt_data)
        receipt_text.config(state=tk.DISABLED)
        receipt_text.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)
        
        # Payment buttons
        btn_frame = ttk.Frame(receipt_window)
        btn_frame.pack(pady=10)
        
        ttk.Button(btn_frame, 
                 text="Proceed to Payment", 
                 command=lambda: self.payment_window(receipt_window)
                 ).pack(side=tk.LEFT, padx=5)
        
        ttk.Button(btn_frame, 
                 text="Close", 
                 command=receipt_window.destroy
                 ).pack(side=tk.RIGHT, padx=5)
        
        # Clear cart after generating receipt
        self.cart.clear()
        cart_window.destroy()

    def payment_window(self, parent_window):
        parent_window.destroy()
        payment_window = tk.Toplevel(self.root)
        payment_window.title("Payment")
        payment_window.geometry("400x300")
        
        ttk.Label(payment_window, 
                text="Select Payment Method", 
                font=("Arial", 14, "bold")
                ).pack(pady=20)
        
        methods = ["Credit/Debit Card", "UPI Payment", "Cash on Delivery"]
        for method in methods:
            ttk.Button(payment_window,
                     text=method,
                     command=lambda m=method: self.process_payment(m, payment_window)
                     ).pack(pady=5, fill=tk.X, padx=50)

    def process_payment(self, method, window):
        window.destroy()
        confirmation_window = tk.Toplevel(self.root)
        confirmation_window.title("Payment Successful")
        confirmation_window.geometry("300x150")
        
        ttk.Label(confirmation_window,
                text="Payment Successful!",
                font=("Arial", 14, "bold")
                ).pack(pady=20)
        
        ttk.Label(confirmation_window,
                text=f"Method: {method}",
                font=("Arial", 12)
                ).pack()
        
        ttk.Button(confirmation_window,
                 text="Close",
                 command=confirmation_window.destroy
                 ).pack(pady=10)
        
        # Clear cart if not already cleared
        self.cart.clear()    
    
    
    def search_product(self):
        search_term = self.search_var.get().strip()
    
        if not search_term:
            messagebox.showwarning("Search", "Please enter a search term")
            return
        
        try:
            # Search in database for products matching the search term
            self.cursor.execute("""
                SELECT id, name, price, category 
                FROM products 
                WHERE name LIKE %s
            """, (f"%{search_term}%",))
            
            results = self.cursor.fetchall()
            
            if not results:
                messagebox.showinfo("Search", "No products found matching your search")
                return
            
            # Create a new window to display search results
            search_window = tk.Toplevel(self.root)
            search_window.title(f"Search Results for '{search_term}'")
            search_window.geometry("600x400")
            
            # Create a frame for the results
            results_frame = tk.Frame(search_window, bg="white")
            results_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
            
            # Add a scrollbar
            scrollbar = ttk.Scrollbar(results_frame)
            scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
            
            # Create a listbox to display results
            results_list = tk.Listbox(
                results_frame, 
                yscrollcommand=scrollbar.set,
                font=("Arial", 12),
                bg="white",
                selectbackground="#3498db",
                selectforeground="white"
            )
            results_list.pack(fill=tk.BOTH, expand=True)
            
            # Configure scrollbar
            scrollbar.config(command=results_list.yview)
            
            # Add products to the listbox
            for product in results:
                product_id, name, price, category = product
                results_list.insert(
                    tk.END, 
                    f"{name} - Rs.{price:.2f} ({category})"
                )
            
            # Add double-click functionality to add to cart
            def on_double_click(event):
                selection = results_list.curselection()
                if selection:
                    selected_product = results[selection[0]]
                    product_id, name, price, category = selected_product
                    
                    # Create a popup to select quantity
                    quantity_window = tk.Toplevel(search_window)
                    quantity_window.title(f"Add {name} to Cart")
                    quantity_window.geometry("300x150")
                    
                    ttk.Label(quantity_window, text="Quantity:").pack(pady=5)
                    quantity_var = tk.IntVar(value=1)
                    ttk.Combobox(
                        quantity_window, 
                        textvariable=quantity_var, 
                        values=list(range(1, 21)),
                        width=5
                    ).pack(pady=5)
                    
                    def add_selected():
                        self.add_to_cart(product_id, name, price, quantity_var)
                        quantity_window.destroy()
                        messagebox.showinfo("Success", f"{name} of  added to cart!")
                    
                    ttk.Button(
                        quantity_window, 
                        text="Add to Cart", 
                        command=add_selected,
                        style="blue.TButton"
                    ).pack(pady=10)
            
            results_list.bind("<Double-Button-1>", on_double_click)
            
            # Add a label showing number of results
            ttk.Label(
                search_window, 
                text=f"Found {len(results)} matching products",
                font=("Arial", 10)
            ).pack(side=tk.BOTTOM, pady=5)
            
        except pymysql.Error as e:
            messagebox.showerror("Database Error", f"Search failed: {str(e)}")
            

    
    def logout(self):
        confirm = messagebox.askyesno("Logout", "Are you sure you want to logout?")
        if confirm:
            self.root.destroy()  # Close the current user panel
            root = tk.Tk()       # Create new root window
            LoginScreen(root, "user")  # Show login screen
            root.mainloop()
         

if __name__ == "__main__":
    root = tk.Tk()
    app = FrontScreen(root)
    root.mainloop() 

pip install mysql-connector-python
